In [48]:
import sys
sys.path.insert(0, "kalman")

import cv2
import numpy as np
import time
import uuid
import imutils
import dlib
from kalman.sort import Sort
import pickle
from playsound import playsound
import _thread as thread

# Subtraction Based Detection With Kalman Tracking

In [49]:
import math

def get_vector_length(vec):
    return math.sqrt(vec[0]**2 + vec[1]**2)

def normalize_vector(vec):
    l = get_vector_length(vec)
    if l == 0:
        return (0,0)
    return (vec[0]/l,vec[1]/l)

In [50]:
def save_all_settings(settings):
    pickle.dump(settings,open('settings.ini', 'wb'))
     
def load_settings():
    try:
        settings = pickle.load(open('settings.ini', 'rb'))
        
    except:
        return None
    return settings

In [51]:
kernelSize = (7,7)

kernel = cv2.getStructuringElement(cv2.MORPH_RECT,kernelSize)
mean_thresh = 5

skip_frames = 0
current_frame = 0

col_scale_ratio = 0.5
authorization_time = 2
enable_landmark_tracking = False

#OFFICE CAM
#area_min_thresh = 10000
#area_max_thresh = 110000
#

# #example_02.mp4
area_min_thresh = 15000
area_max_thresh = 200000

trackers = []
labels = []
max_age = 10
min_hits = 5
confidence = 0.3
tracker =  Sort(use_dlib= False,max_age=max_age,min_hits=min_hits)

markers = [(735,251)]
marker_radius = 10
collision_radius = 80

positions = {}

entrance_p1_x = 0
entrance_p1_y = 0
entrance_p2_x = 100
entrance_p2_y = 100

exit_p1_x = 100
exit_p1_y = 100
exit_p2_x = 200
exit_p2_y = 200


mouseX = 0
mouseY = 0

save_settings = False

In [52]:
def mouse_callback(event, x,y,flags, params):
    global markers
    global mouseX,mouseY
    global save_settings
    if event == cv2.EVENT_LBUTTONDBLCLK:
        save_settings = True
        markers[0] = (x,y)
        print("Node moved to position " +  str(x) + "," + str(y))
    elif event == cv2.EVENT_MOUSEMOVE:
        mouseX = x
        mouseY = y

In [53]:
def intersect(x1,y1, k, x, min_y, max_y):
    y = y1 + (x - x1) * k
    return y >= min_y and y <= max_y


def collision_detection(x1,y1,x2,y2,r_x,r_y,r_width,r_height):
    if x1 == x2:
        print(1)
        return x1 >= r_x and x1 < r_x+r_width
  
    elif y1 == y2:
        print(2)
        return y1 >= r_y and y1 < r_y+r_height
  

    k = (y2 - y1) / (x2 - x1)

    if intersect(x1, y1, k, r_x, r_y, r_y+r_height-1):
        print(3)
        return True
  

    if intersect(x1, y1, k, r_x+r_width-1, r_y, r_y+r_height-1):
        print(4)
        return True
  

    k_inv = (x2 - x1) / (y2 - y1)

    if intersect(y1, x1, k_inv, r_y, r_x, r_x+r_width-1):
        print(5)
        return True
  
  
    if intersect(y1, x1, k_inv, r_y+r_height-1, r_x, r_x+r_width-1):
        print(6)
        return True
  

    return False

In [54]:
def line_intersection(line1, line2):
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
       return False

    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return True

In [55]:
def line_intersect(Ax1, Ay1, Ax2, Ay2, Bx1, By1, Bx2, By2):
    """ returns a (x, y) tuple or None if there is no intersection """
    d = (By2 - By1) * (Ax2 - Ax1) - (Bx2 - Bx1) * (Ay2 - Ay1)
    if d:
        uA = ((Bx2 - Bx1) * (Ay1 - By1) - (By2 - By1) * (Ax1 - Bx1)) / d
        uB = ((Ax2 - Ax1) * (Ay1 - By1) - (Ay2 - Ay1) * (Ax1 - Bx1)) / d
    else:
        return False
    if not(0 <= uA <= 1 and 0 <= uB <= 1):
        return False
    x = Ax1 + uA * (Ax2 - Ax1)
    y = Ay1 + uA * (Ay2 - Ay1)
 
    return True


def rect_line_intersect(rect,line):
    (x_min,y_min,x_max,y_max) = rect
    (x1,y1,x2,y2) = line
    
    tl = (x_min,y_min)
    tr = (x_max,y_min)
    bl = (x_min,y_max)
    br = (x_max,y_max)
    
    #top line intersect
    intersect = line_intersect(x1,y1,x2,y2,tl[0],tl[1],tr[0],tr[1])
    if intersect:
        return True
    
    #bottom line intersect
    intersect = line_intersect(x1,y1,x2,y2,bl[0],bl[1],br[0],br[1])
    if intersect:
        return True
    
    #left line intersect
    intersect = line_intersect(x1,y1,x2,y2,tl[0],tl[1],bl[0],bl[1])
    if intersect:
        return True
        
    #right line intersect
    intersect = line_intersect(x1,y1,x2,y2,tr[0],tr[1],br[0],br[1])
    if intersect:
        return True
    
    return False

In [56]:
def scale_rect(x_min,y_min,x_max,y_max,ratio):
    w = x_max - x_min
    h = y_max - y_min
    half_ration = ratio / 2
    
    x_min += (ratio * x_min)
    x_max 

In [57]:
def calculateDistance(x1,y1,x2,y2):  
     dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)  
     return dist  

In [58]:
def play_sound(file):
    playsound(file)

In [59]:

camera_url = r".\videos\office_clipped.mp4"
import time

backSub = cv2.createBackgroundSubtractorKNN()

vs = cv2.VideoCapture(camera_url)
time.sleep(1)

windowname = "Video Stream"

cv2.namedWindow(windowname)
cv2.setMouseCallback(windowname, mouse_callback)


entrance_line_first = True
exit_line_first = True

in_place = {}
out_place = {}
ids_dirs = {}
landmark_users = {}
allowed_users = []
in_count = 0

action_lines = []
normals = []
crossed_users = []

line_length_modifier = 3

settings = load_settings()
if settings is not None:
    entrance_p1_x = settings["entrance_p1_x"]
    entrance_p1_y = settings["entrance_p1_y"]
    entrance_p2_x = settings["entrance_p2_x"]
    entrance_p2_y = settings["entrance_p2_y"]
    markers = settings["markers"]
    collision_radius = settings["collision_radius"]
    action_lines =  settings["action_lines"]
    normals = settings["normals"]
    
while True:
#    time.sleep(0.05)
    invalid_ids = []
    ret,frame = vs.read()
    if ret == False:
        print("Unable to open streaming source.")
        break
    if frame is None or len(frame) == 0:
        continue
    
    current_frame += 1
    if current_frame <= skip_frames:
        continue
    else:
        current_frame = 0
    
    frame = imutils.resize(frame,width = 800)
    orig = frame.copy()
    frame = cv2.GaussianBlur(frame,(21,21),0) 
    fgMask = backSub.apply(frame)
    
    (T,thresh) = cv2.threshold(fgMask,25,255,cv2.THRESH_BINARY)
    
    cnts = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    
    rgb = cv2.cvtColor(orig,cv2.COLOR_BGR2RGB)     
     
    dets = []
    probs = []
    
    for cnt in cnts:
        (x,y,w,h) = cv2.boundingRect(cnt)
        (startX,startY,endX,endY) = (x,y,x+w,y+h)
        area = w*h
        
        if area > area_min_thresh and area < area_max_thresh:
            dets.append([startX, startY, endX, endY,1])

    dets = np.array(dets)    
    trackers = tracker.update(dets,rgb)   
    
    ids = []

       
    for t in trackers:
        startX = int(t[0])
        startY = int(t[1])
        endX = int(t[2])
        endY = int(t[3])
        id = int(t[4])
        
        ids.append(id)
        
        centeroid = (int((endX + startX)/2),int((endY + startY)/2))
        cv2.circle(orig,centeroid,4,(0,255,0),-1)
        arr = positions.get(id)
        if arr is None:
            arr = []
        arr.append(centeroid)
        positions[id] = arr
        
        ##Check circle collision
        if enable_landmark_tracking:
            for (x,y) in markers:
                user_landmark_dist = calculateDistance(centeroid[0],centeroid[1],x,y)

                if user_landmark_dist <= collision_radius:
                    col_id = landmark_users.get(id)
                    if col_id is None:
                        landmark_users[id] = time.time()
                    else:
                        t = landmark_users[id]
                        if time.time() - t >= authorization_time:
                            if id in allowed_users:
                                pass
                            else:
                                allowed_users.append(id)
        ########
        
        for i in range(0,len(arr)):
            pos1 = arr[i]
            if i < len(arr)-1:
                pos2 = arr[i+1]
                cv2.line(orig,pos1,pos2,(0,255,255),2)
        
        for i,(x1,y1,x2,y2) in enumerate(action_lines):
            (_x1,_y1,_x2,_y2) = (x1,y1,x2,y2) 
            #Calculate the normal
            (x1_c, y1_c) = int((x1 + x2)/2),int((y1 + y2)/2)     
            (x2_c,y2_c) = (x1_c-int(y2/line_length_modifier)+int(y1/line_length_modifier),y1_c + int(x2/line_length_modifier)-int(x1/line_length_modifier))
            (x1_c, y1_c) =(x1_c+int(y2/line_length_modifier)-int(y1/line_length_modifier),y1_c - int(x2/line_length_modifier)+int(x1/line_length_modifier))
            entrance_dir_vec = normalize_vector((x2_c-x1_c,y2_c-y1_c))

            (x1,y1,x2,y2) = (x1_c,y1_c,x2_c,y2_c)
            
            entr_intersect = rect_line_intersect((startX,startY,endX,endY),(x1,y1,x2,y2))
            if entr_intersect:
                #calculate the vector between the bounding box center and entrance line center
                line_center = (int((x1 + x2)/2),int((y1 + y2)/2))
                vec_line_to_rect = normalize_vector((centeroid[0] - line_center[0] , centeroid[1] - line_center[1]))
                dot = np.dot(vec_line_to_rect,normalize_vector((_x2-line_center[0],_y2-line_center[1])))
                angle = abs(math.degrees(np.arccos(dot)))

                if angle < 90:
                    id_dir = ids_dirs.get(id)
                    if id_dir is None: #First time registered
                        pass
                    else:
                        if id_dir == 0:
                            in_count = max(0, in_count - 1)
                    ids_dirs[id] = 1

                else:
                    id_dir = ids_dirs.get(id)
                    if id_dir is None: #First time registered
                        pass
                    else:
                        if id_dir == 1:
                            in_count += 1
                        ids_dirs[id] = 0

                break
        
        rect_color = (255,0,0)
        if ids_dirs.get(id) == 0:
            invalid_ids.append(id)
            thread.start_new_thread(play_sound,("audio/alarm.wav",))
            rect_color = (0,0,255)
            
        cv2.rectangle(orig,(startX,startY),(endX,endY),rect_color,5)
        cv2.putText(orig,str(id) + "," ,(startX ,startY),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,255))

    positions = {i:j for (i,j) in positions.items() if i in ids}
    
    if enable_landmark_tracking:
        for (x,y) in markers:
            cv2.circle(orig,(x,y),marker_radius,(0,255,255),-1)
            cv2.circle(orig,(x,y),int(collision_radius/2),(200,200,160),2)
            cv2.circle(orig,(x,y),collision_radius,(200,200,160),5)
    
    message = ""
    if len(invalid_ids)>0:
        message = "Moevement violations detected for users: "
    for id in invalid_ids:
        message+=  str(id)+" , "
    
    cv2.putText(orig,message,(50,50),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(0,0,255))
    
    
    #draw all action lines
    for (x1,y1,x2,y2) in action_lines:
        cv2.arrowedLine(orig,(x1,y1),(x2,y2),(0,255,0),3)
        (x1_c, y1_c) = int((x1 + x2)/2),int((y1 + y2)/2)  

        (x2_c,y2_c) = (x1_c-int(y2/line_length_modifier)+int(y1/line_length_modifier),y1_c + int(x2/line_length_modifier)-int(x1/line_length_modifier))
        (x1_c, y1_c) =(x1_c+int(y2/line_length_modifier)-int(y1/line_length_modifier),y1_c - int(x2/line_length_modifier)+int(x1/line_length_modifier))
        entrance_dir_vec = normalize_vector((x2_c-x1_c,y2_c-y1_c))
        cv2.line(orig,(x1_c,y1_c),(x2_c,y2_c),(125,0,125),2)      

    
    cv2.imshow(windowname,orig)
    cv2.imshow('Thresh', thresh)

    key = cv2.waitKey(1)
    
    if key == ord("q") or key == ord("Q"):
        break
        
    elif key == ord("p") or key == ord("P"):
        cv2.waitKey(0)
    
    elif key == ord("e") or key == ord("E"):
        save_settings = True
        if entrance_line_first:
            entrance_line_first = False
            entrance_p1_x = mouseX
            entrance_p1_y = mouseY
            entrance_p2_x = mouseX
            entrance_p2_y = mouseY
        else:
            entrance_line_first = True
            entrance_p2_x = mouseX
            entrance_p2_y = mouseY       
            action_lines.append((entrance_p1_x,entrance_p1_y,entrance_p2_x,entrance_p2_y))
            
            (x1, y1) = int((entrance_p1_x + entrance_p2_x)/2),int((entrance_p1_y + entrance_p2_y)/2)    
            (x2,y2) = (x1-entrance_p2_y+entrance_p1_y,y1 + entrance_p2_x-entrance_p1_x)
            entrance_dir_vec = normalize_vector((x2-x1,y2-y1))
            normals.append(entrance_dir_vec)
            
    elif key == ord("x") or key == ord("X"):
        if len(action_lines)>0:
            del action_lines[-1]
            del normals[-1]
            save_settings = True

            
    elif  key == ord("+"):
        save_settings = True
        collision_radius += 10
        
    elif key == ord("-"):
        save_settings = True
        collision_radius = max(marker_radius,collision_radius - 10)
    
    if save_settings:
        print("Saving settings...")
        settings = {}
        settings["entrance_p1_x"] = entrance_p1_x
        settings["entrance_p1_y"] = entrance_p1_y
        settings["entrance_p2_x"] = entrance_p2_x
        settings["entrance_p2_y"] = entrance_p2_y
        settings["exit_p1_x"] = exit_p1_x 
        settings["exit_p2_x"] = exit_p2_x        
        settings["exit_p2_y"] = exit_p2_y
        settings["markers"] = markers
        settings["collision_radius"] = collision_radius
        settings["action_lines"] = action_lines
        settings["normals"] = normals
        save_settings = False
        save_all_settings(settings)

vs.release()
cv2.destroyAllWindows()